# Generate the sentences-tags csv to train the Bert model

! pip install strsimpy  
! pip install pyspellchecker  

In [1]:
import pandas as pd
import numpy as np
import random
import re
import gensim
from gensim.models import Word2Vec,KeyedVectors
from strsimpy.jaro_winkler import JaroWinkler
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import itertools

##### Libraries
! pip install pyspellchecker  
nltk.download('stopwords')

!wget -P ./data -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [2]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [3]:
model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)

In [4]:
prevoc = get_prevoc()
voc = get_voc(model_w2v, prevoc)
gen = get_gen()
tagger = get_tagger()
structures = get_structures()

print(verif_dict_generator(gen))
print(verif_dict_tagger(tagger))
print(verif_dict_tagger_links_gen(tagger, gen))
print(verif_dict_tagger_links_voc(tagger, voc))
print(verif_structure_link_tag(structures, tagger, display=False))

True
True
True
True
True


In [5]:
all_tags = get_all_tags()
print(all_tags)

['O', 'STAT_B', 'STAT_I', 'STAT_E', 'MANU_B', 'MANU_I', 'MANU_E', 'AIRP_B', 'AIRP_I', 'AIRP_E', 'AIRL_B', 'AIRL_I', 'AIRL_E', 'COUN_B', 'COUN_I', 'COUN_E', 'DATE1_B', 'DATE1_I', 'DATE1_E', 'DATE2_B', 'DATE2_I', 'DATE2_E', 'STUDIED_B', 'STUDIED_I', 'STUDIED_E']


In [6]:
voc_stopwords = set(stopwords.words('english'))

In [7]:
select_from_sentence_gen(gen, "#COMP#good #client_satisfaction")[1:]

'prodigious_customers_satisfaction'

In [8]:
structure = structures[np.random.choice(len(structures),1)[0]]
init_structure(structure)

['#STATS#wordcloud']

In [9]:
structure = structures[np.random.choice(len(structures),1)[0]]
structure_init = init_structure(structure)
sent,tags = generete_sentence_from_structure(tagger,structure_init)
print("Same length :", len(sent.split()) == len(tags.split()))
sent,tags

Same length : True


('display us the plan of plane in JORDAN from the end of 1982 to APRIL 2002 for AIRBUS , CESSNA',
 'O O O STAT_B STAT_I STAT_E O COUN_B O O O O DATE1_B O DATE2_B DATE2_E O MANU_B O MANU_B')

In [10]:
sentence = 'I woulde lirke the graph of the numbr of fligt for Boing on mondy 2020'
auto_correction(model_w2v.vocab, voc, voc_stopwords, sentence)

['I',
 'would',
 'like',
 'the',
 'graph',
 'of',
 'the',
 'number',
 'of',
 'flight',
 'for',
 'Boeing',
 'on',
 'monday',
 '2020']

##### generation of the datasets

In [11]:
train_df = generate_dataframe_for_bert(n=10000)
eval_df = generate_dataframe_for_bert(n=1000)

In [12]:
train_df.to_csv(data_directory+'tagger_train.csv', sep='§', encoding='utf-8')
eval_df.to_csv(data_directory+'tagger_eval.csv', sep='§', encoding='utf-8')